In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client langchain-google-genai



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from memoryStore import (
    init_memory,
    save_email_memory,
    get_sender_history,
    get_conn
)
init_memory()


In [3]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import datetime


In [5]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = [
    'https://www.googleapis.com/auth/gmail.modify',
    'https://www.googleapis.com/auth/calendar'
]


def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials1.json', SCOPES
            )
            creds = flow.run_local_server(port=0)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    gmail_service = build('gmail', 'v1', credentials=creds)
    calendar_service = build('calendar', 'v3', credentials=creds)

    return gmail_service, calendar_service


service, calendar_service = get_gmail_service()


In [6]:
def create_event(calendar_service, summary, start_dt, end_dt, attendees=None):
    if not summary or not start_dt or not end_dt:
        print("Skipping event creation: missing required info")
        return None

    event = {
        "summary": summary,
        "start": {"dateTime": start_dt, "timeZone": "Asia/Kolkata"},
        "end": {"dateTime": end_dt, "timeZone": "Asia/Kolkata"},
    }

    if attendees:
        event["attendees"] = [{"email": email} for email in attendees]

    try:
        created_event = calendar_service.events().insert(
            calendarId='primary', body=event).execute()
        print("Event created:", created_event.get('htmlLink'))
        return created_event
    except HttpError as error:
        print("Calendar API error:", error)
        return None


In [7]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'  
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


In [8]:
def read_inbox(service, max_results=5):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=max_results
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_id = msg['id']
        message = service.users().messages().get(
            userId='me',
            id=msg_id,
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = message['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📩 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))


In [9]:
read_inbox(service)


📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Follow-Up on Meeting Scheduling

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: request fo meeting scheduling

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Meeting Request

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: lets connect

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Project Discussion Meeting


In [10]:

import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env
load_dotenv()

# Read API key
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

# Create OpenAI client
client = OpenAI(api_key=api_key)

try:
    # List models available for this key
    models = client.models.list()

    print("Models available for this key:")
    for model in models.data:
        print(model.id)

except Exception as e:
    print("Error:", e)


Models available for this key:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano

In [11]:
! pip install langchain langgraph langchain-openai python-dotenv 



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!pip install -U langchain-openai


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install google-generativeai python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

# Load env
load_dotenv()

API_KEY = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")

if not API_KEY:
    raise ValueError("No Gemini / Google API key found")

genai.configure(api_key=API_KEY)

print("📋 Available models for this API key:\n")

for model in genai.list_models():
    print(f"Model name : {model.name}")
    print(f"Supports generateContent: {'generateContent' in model.supported_generation_methods}")
    print("-" * 50)


C:\Users\fatim\AppData\Local\Temp\ipykernel_22408\2851657525.py:3: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


📋 Available models for this API key:

Model name : models/gemini-2.5-flash
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-2.5-pro
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-2.0-flash
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-2.0-flash-001
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-2.0-flash-exp-image-generation
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-2.0-flash-lite-001
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-2.0-flash-lite
Supports generateContent: True
--------------------------------------------------
Model name : models/gemini-exp-1206
Supports generateContent: True
-------------------------

In [15]:
!ollama run phi3

^C


In [ ]:
from langchain_community.chat_models import ChatOllama

def load_llm1():
    return ChatOllama(model="phi3", temperature=0)


In [16]:
def load_llm():
    from dotenv import load_dotenv
    import os
    from langchain_google_genai import ChatGoogleGenerativeAI

    load_dotenv()

    # LangChain prefers GOOGLE_API_KEY
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")

    if not GOOGLE_API_KEY:
        raise ValueError("Please set GOOGLE_API_KEY or GEMINI_API_KEY in .env")

    llm = ChatGoogleGenerativeAI(
        model="gemini-3-flash-preview",   # ✅ WORKING MODEL
        temperature=0,
        google_api_key=GOOGLE_API_KEY
    )

    return llm


In [ ]:
from langchain_openai import ChatOpenAI

def load_llm1():
    return ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)


In [17]:
load_llm()

ChatGoogleGenerativeAI(profile={}, google_api_key=SecretStr('**********'), model='gemini-3-flash-preview', temperature=0.0, client=<google.genai.client.Client object at 0x000002ACC1197260>, default_metadata=(), model_kwargs={})

In [19]:
def build_email_prompt(user_prompt: str):
    from langchain_core.prompts import (
        ChatPromptTemplate,
        SystemMessagePromptTemplate,
        HumanMessagePromptTemplate
    )
    from datetime import datetime

    current_date = datetime.today().strftime("%A, %B %d, %Y")

    system_text = f"""
You are an AI email intent classification and response agent.
Today's date is {current_date}.

USER INSTRUCTIONS:
{user_prompt}

CRITICAL RULES (MUST FOLLOW):
1. You MUST respond in STRICT JSON only.
2. Do NOT use markdown, explanations, or extra text.
3. Grammar mistakes, short emails, or informal language
   do NOT mean an email should be ignored.

INTENT RULES (VERY IMPORTANT):
- If the subject OR body mentions ANY of the following:
  meeting, schedule, call, discussion, interview, appointment

  → You MUST NOT return "ignore".

- Emails asking for meetings are ALWAYS genuine.
- Promotions, newsletters, and marketing emails
  are the ONLY emails you should ignore.

ACTION DECISION:
- Use "schedule" ONLY if date and time are clearly mentioned.
- Otherwise use "reply" and ask for availability.
- If unsure, ALWAYS choose "reply".

OUTPUT JSON SCHEMA (NO EXCEPTIONS):
{{{{ 
  "action": "reply" | "schedule" | "ignore",
  "reply": "string",
  "event": {{{{
    "title": "string",
    "date": "YYYY-MM-DD",
    "start_time": "HH:MM",
    "end_time": "HH:MM"
  }}}}
}}}}

REPLY STYLE RULES:
- Start replies with a short thank-you.
- Be polite and professional.
"""

    system_msg = SystemMessagePromptTemplate.from_template(system_text)

    human_msg = HumanMessagePromptTemplate.from_template(
        "Subject: {subject}\nFrom: {sender}\nBody: {body}"
    )

    return ChatPromptTemplate.from_messages([system_msg, human_msg])


In [20]:
import base64
from email.mime.text import MIMEText

def fetch_unread_emails(service, max_results=5):
    return service.users().messages().list(
        userId="me",
        labelIds=["UNREAD"],
        maxResults=max_results
    ).execute().get("messages", [])

def extract_email(service, msg_id):
    msg = service.users().messages().get(
        userId="me",
        id=msg_id,
        format="full"
    ).execute()

    headers = msg["payload"]["headers"]
    subject = sender = ""

    for h in headers:
        if h["name"] == "Subject":
            subject = h["value"]
        elif h["name"] == "From":
            sender = h["value"]

    body = ""
    payload = msg["payload"]

    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"].get("data")
                if data:
                    body = base64.urlsafe_b64decode(data).decode()
    else:
        data = payload["body"].get("data")
        if data:
            body = base64.urlsafe_b64decode(data).decode()

    return subject, sender, body, msg["threadId"]

import re

def clean_email_body(body: str) -> str:
    # Remove markdown JSON blocks
    body = re.sub(r"```json[\s\S]*?```", "", body, flags=re.IGNORECASE)

    # Remove quoted replies (Gmail style)
    body = re.split(r"\nOn .* wrote:", body)[0]

    # Remove lines starting with >
    body = "\n".join(
        line for line in body.splitlines()
        if not line.strip().startswith(">")
    )

    return body.strip()

def send_reply(service, to, subject, reply, thread_id):
    message = MIMEText(reply)
    message["to"] = to
    message["subject"] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().messages().send(
        userId="me",
        body={"raw": raw, "threadId": thread_id}
    ).execute()

def mark_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()


In [21]:
import json

def process_all_unread_emails(service, user_prompt, calendar_service=None, max_results=20):
    """
    Process unread emails using AI and take actions (reply, schedule, ignore).
    """
    # Build prompt and LLM
    prompt = build_email_prompt(user_prompt)
    llm = load_llm()

    # Fetch unread emails
    emails = fetch_unread_emails(service, max_results=max_results)
    print(f"\n📬 Total unread emails: {len(emails)}\n")

    if not emails:
        print("🎉 No unread emails.")
        return

    for idx, msg in enumerate(emails, start=1):
        print("=" * 70)
        print(f"📧 Processing email {idx}/{len(emails)}")

        # Extract email details
        subject, sender, body, thread_id = extract_email(service, msg["id"])
        body = clean_email_body(body)

        print(f"From   : {sender}")
        print(f"Subject: {subject}")
        print("Body preview:")
        print(body[:300])
        print("=" * 70)

        # ---- AI RESPONSE ----
        try:
            # 1. Invoke the LLM
            response_obj = llm.invoke(
                prompt.format_prompt(
                    subject=subject,
                    sender=sender,
                    body=body,
                    action="Reply"  
                ).to_messages()
            )
            
            # 2. FIX: Handle content if it is a list or a string
            ai_response = response_obj.content
            if isinstance(ai_response, list):
                # Extract text if it's a list of parts
                ai_response = "".join([part['text'] if isinstance(part, dict) else str(part) for part in ai_response])
            
            # 3. FIX: Clean Markdown formatting if the AI added it
            ai_response = ai_response.strip().replace("```json", "").replace("```", "").strip()

        except Exception as e:
            print(f"❌ AI invocation failed: {e}. Skipping email.")
            continue

        # ---- PARSE JSON ----
        try:
            result = json.loads(ai_response)
        except json.JSONDecodeError:
            print("❌ AI returned invalid JSON. Skipping email.")
            print(f"DEBUG: Raw AI Output was: {ai_response}")
            continue

        action = result.get("action", "ignore")
        reply = result.get("reply", "")

        # ---- Handle scheduled event ----
        if action == "schedule":
            if calendar_service is None:
                print("❌ Calendar service not provided. Cannot schedule event.")
                continue

            event = result.get("event", {})
            start_dt = f"{event.get('date','')}T{event.get('start_time','00:00')}:00"
            end_dt = f"{event.get('date','')}T{event.get('end_time','00:00')}:00"

            create_event(
                calendar_service,
                title=event.get("title", "Meeting"),
                description=body,
                start_dt=start_dt,
                end_dt=end_dt
            )

            mark_read(service, msg["id"])
            print("✅ Event scheduled & email marked as read")
            continue

        # ---- Handle ignored email ----
        if action == "ignore":
            print("🚫 AI chose to ignore this email.")
            continue

        # ---- Draft reply ----
        print("\n🤖 AI Draft Reply")
        print("-" * 50)
        print(reply)
        print("-" * 50)

        decision = input("Send this reply? (y / edit / skip): ").lower()

        if decision == "y":
            send_reply(service, sender, subject, reply, thread_id)
            mark_read(service, msg["id"])
            print("✅ Reply sent & marked as read")

        elif decision == "edit":
            edited = input("\n✏️ Enter your edited reply:\n")
            send_reply(service, sender, subject, edited, thread_id)
            mark_read(service, msg["id"])
            print("✅ Edited reply sent & marked as read")

        else:
            print("⏭️ Skipped (email remains unread)")

In [ ]:
USER_PROMPT = """
Reply only to genuine emails.
Ignore promotions and job ads.
Be polite, concise, and human-like.
Do not add name or signature.
"""

process_all_unread_emails(service, USER_PROMPT)



📬 Total unread emails: 1

📧 Processing email 1/1
From   : fatimi bee <fatimibee312@gmail.com>
Subject: Project Discussion Meeting
Body preview:
Hi Saniya,

I hope you are doing well. Can we schedule a meeting tomorrow at 3:00 PM
IST to discuss the Infosys Email Assistant project progress?
Please let me know if that time works for you or suggest another slot.

Thanks,
Fatimi

🤖 AI Draft Reply
--------------------------------------------------
Yes, tomorrow at 3:00 PM IST works for me. I look forward to discussing the project progress.
--------------------------------------------------
✅ Reply sent & marked as read


In [22]:
def get_state_schema():
    from typing import TypedDict, Optional, Dict

    class EmailState(TypedDict):
        subject: str
        sender: str
        body: str
        thread_id: str
        decision: Dict
        final_reply: Optional[str]

    return EmailState


In [23]:
def ai_decision_node(state, prompt, llm):
    import json
    import re

    # 🔁 FETCH MEMORY
    history = get_sender_history(state["sender"])
    memory_text = ""

    if history:
        memory_text = "\nPAST INTERACTIONS WITH THIS SENDER:\n"
        for action, ts in history:
            memory_text += f"- {action} at {ts}\n"

    chain = prompt | llm
    response = chain.invoke({
        "subject": state["subject"],
        "sender": state["sender"],
        "body": state["body"] + memory_text
    })

    # Normalize content
    content = response.content
    if isinstance(content, list):
        raw = "".join(
            part if isinstance(part, str) else part.get("text", "")
            for part in content
        )
    else:
        raw = content

    raw = raw.strip()
    raw = re.sub(r"^```json\s*|```$", "", raw, flags=re.MULTILINE)

    try:
        decision = json.loads(raw)
    except Exception:
        decision = {
            "action": "reply",
            "reply": "Thank you for your email. I’ll get back to you shortly.",
            "event": {}
        }

    # safety defaults
    decision.setdefault("action", "reply")
    decision.setdefault("reply", "")
    decision.setdefault("event", {})

    return {"decision": decision}


In [24]:
def human_confirmation_node(state):
    reply = state["decision"].get("reply")

    print("\n🤖 AI Draft Reply")
    print("-" * 60)
    print(reply)
    print("-" * 60)

    while True:
        choice = input("Send reply? (y / n / edit): ").strip().lower()

        if choice == "y":
            return {"final_reply": reply}

        if choice == "n":
            return {"final_reply": None}

        if choice == "edit":
            print("\n✏️ Enter edited reply (empty line to finish):")
            lines = []
            while True:
                line = input()
                if not line:
                    break
                lines.append(line)
            return {"final_reply": "\n".join(lines)}


In [30]:
def calendar_node_factory(calendar_service):
    def calendar_node(state):
        event = state.get("decision", {}).get("event", {})

        if not event.get("date") or not event.get("start_time"):
            print("⚠️ Incomplete event data. Skipping calendar.")
            return {}

        try:
            start_dt = f"{event['date']}T{event['start_time']}:00"
            end_dt = f"{event['date']}T{event.get('end_time', event['start_time'])}:00"

            create_event(
                calendar_service,
                summary=event.get("title", "Meeting"),
                description=state.get("body", ""),
                start=start_dt,
                end=end_dt
            )

            save_email_memory(
                sender=state["sender"],
                subject=state["subject"],
                thread_id=state["thread_id"],
                action="schedule"
            )

            print("🗓️ Event scheduled & remembered")

        except Exception as e:
            print(f"❌ Calendar Error: {e}")

        return {}

    return calendar_node


In [26]:
def send_node_factory(service):
    def send_node(state):
        if state.get("final_reply"):
            send_reply(
                service,
                state["sender"],
                state["subject"],
                state["final_reply"],
                state["thread_id"]
            )

            # 💾 SAVE TO MEMORY
            save_email_memory(
                sender=state["sender"],
                subject=state["subject"],
                thread_id=state["thread_id"],
                action="reply",
                reply=state["final_reply"]
            )

            print("✅ Reply sent & saved to memory")
        else:
            print("⏭️ Reply skipped")

        return {}

    return send_node


In [27]:
def build_email_graph(user_prompt: str, service, calendar_service):
    from langgraph.graph import StateGraph, END

    EmailState = get_state_schema()
    llm = load_llm()
    prompt = build_email_prompt(user_prompt)

    graph = StateGraph(EmailState)

    graph.add_node("ai_decision", lambda s: ai_decision_node(s, prompt, llm))
    graph.add_node("human_confirm", human_confirmation_node)
    graph.add_node("send", send_node_factory(service))
    graph.add_node("calendar", calendar_node_factory(calendar_service))

    graph.set_entry_point("ai_decision")

    def router(state):
        return state.get("decision", {}).get("action", "reply")

    graph.add_conditional_edges(
        "ai_decision",
        router,
        {
            "reply": "human_confirm",
            "schedule": "calendar",
            "ignore": END
        }
    )

    graph.add_edge("human_confirm", "send")
    graph.add_edge("send", END)
    graph.add_edge("calendar", END)

    return graph.compile()


In [28]:
def run_email_assistant(service, calendar_service, user_prompt, max_results=5):
    app = build_email_graph(user_prompt, service, calendar_service)
    emails = fetch_unread_emails(service, max_results=max_results)

    if not emails:
        print("📭 No unread emails found.")
        return

    print(f"\n📬 Total unread emails: {len(emails)}")

    for i, msg in enumerate(emails, start=1):
        print("\n" + "=" * 70)
        print(f"📧 Processing email {i}/{len(emails)}")

        subject, sender, body, thread_id = extract_email(service, msg["id"])
        body = clean_email_body(body)

        print(f"From   : {sender}")
        print(f"Subject: {subject}")
        print("\n📨 Email Body")
        print("-" * 60)
        print(body.strip())
        print("-" * 60)

        state = {
            "subject": subject,
            "sender": sender,
            "body": body,
            "thread_id": thread_id,
            "decision": {},
            "final_reply": None
        }

        try:
            result = app.invoke(state)

            if result.get("final_reply") or result.get("decision", {}).get("action") != "ignore":
                mark_read(service, msg["id"])
                print("📩 Email marked as read")
            else:
                print("⏸️ Email left unread")

        except Exception as e:
            print(f"❌ Graph failed: {e}")


In [33]:
USER_PROMPT = """
Reply only to genuine emails.
Ignore promotions and job ads.
Schedule meetings if the email discusses a meeting, call, interview, or appointment.
"""

# FIX: Pass BOTH service and calendar_service to the runner
run_email_assistant(
    service=service, 
    calendar_service=calendar_service, 
    user_prompt=USER_PROMPT,
    max_results=5
)


📬 Total unread emails: 1

📧 Processing email 1/1
From   : fatimi bee <fatimibee312@gmail.com>
Subject: Decide the project discussion time.

📨 Email Body
------------------------------------------------------------
Hi Saniya,
Please let me know when we discuss the project idea before 5 february 2026.
------------------------------------------------------------

🤖 AI Draft Reply
------------------------------------------------------------
Thank you for your email. I would be happy to discuss the project idea with you. Please let me know your availability so we can schedule a time before February 5th.
------------------------------------------------------------
✅ Reply sent & saved to memory
📩 Email marked as read


In [35]:
conn = get_conn()
cur = conn.cursor()

for row in cur.execute("SELECT sender, action,reply, created_at FROM email_memory"):
    print(row)

conn.close()


('fatimi bee <fatimibee312@gmail.com>', 'reply', 'Thank you for your email. I would be happy to discuss the project idea with you. Please let me know your availability so we can schedule a time before February 5th.', '2026-01-30T12:20:00.915827')
